<h1 align="center">Analítica de datos para la toma de decisiones empresariales</h1>
<h1 align="center">Ejemplo: Identificación de datos raros multivariantes</h1>
<h1 align="center">Centro de Educación Continua</h1>
<h1 align="center">EAFIT</h1>
<h1 align="center">2023</h1>
<h1 align="center">MEDELLÍN - COLOMBIA </h1>

<table>
 <tr align=left><td><img align=left src="https://github.com/carlosalvarezh/CFD_Applied/blob/master/figs/CC-BY.png?raw=true">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license.(c) Carlos Alberto Alvarez Henao</td>
</table>

## Introducción

El [dataset de Pokémon](https://www.kaggle.com/datasets/rounakbanik/pokemon?select=pokemon.csv) es un conjunto de datos que recopila información sobre diferentes especies de Pokémon, que son criaturas ficticias utilizadas en la serie de videojuegos, programas de televisión y otros medios relacionados. Este conjunto de datos es comúnmente utilizado en ejemplos de análisis multivariante y análisis de datos en general.

Cada fila en el dataset representa a un Pokémon individual y contiene múltiples variables (atributos) que describen diferentes aspectos de cada especie. 

Se explorarán varios modelos centrados tanto en métodos de detección de anomalías univariados como multivariados. Los primeros se centran en una única variable y los últimos en combinaciones de variables. 

- Extreme Value Analysis
- Z-score method
- K-means clustering method
- Local Outlier Factor
- Isolation forest
- DBScan
- One Class SVM


## Análisis Univariado

Lo primero que haremos es cargar los datos y  hacer una exploración de los mismos

In [ ]:
#load necessary libraries for the data analysis
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import os

from scipy.cluster.vq import kmeans
from scipy.cluster.vq import vq

import plotly.express as px

from sklearn.ensemble import IsolationForest

In [ ]:
#dataset available from: https://www.kaggle.com/rounakbanik/pokemon?select=pokemon.csv

pokemon = pd.read_csv("Data/pokemon.csv",
                     usecols = ["name", "type1", "type2", 
                               "base_total", "hp", "attack", "defense",
                               "sp_attack", "sp_defense", "speed", 
                               "generation", "is_legendary", "pokedex_number"])

pokemon = pokemon[["pokedex_number", "name", "type1", "type2", 
                               "base_total", "hp", "attack", "defense",
                               "sp_attack", "sp_defense", "speed", 
                               "generation", "is_legendary"]]

In [ ]:
#use info to show summary measures for all columns inc datatypes
pokemon.info()

esto muestra que tenemos 801 Pokémon en el conjunto de datos con información completa para todos, excepto para el `type2`. Esto no es un problema ya que no se espera que todos los Pokémon tengan un segundo tipo y tener toda la demás información significa que no falta nada.

También podemos ver que las columnas numéricas son: `base_total`, `hp`, `attack`, `defense`, `sp_attak`, `sp_defense`, `speed` y `generation`. Podemos inspeccionarlos usando la función de `describe()`:

In [ ]:
#describe the numerical columns
pokemon.describe()

A partir de esto, podemos ver aproximadamente que deberíamos esperar algunas distribuciones a la derecha con algunas colas largas, ya que para la mayoría de las categorías, el valor del percentíl $75\%$ y el valor máximo es mucho mayor que para otros cuantiles dentro de la distribución. Podemos explorar esto con más detalle más adelante.

Antes de eso, queremos observar algunos de los datos que se encuentran al incio, al final final y de una muestra aleatoria para ver si alguno de los datos es diferente de lo que esperaríamos:

In [ ]:
pokemon.head(10)

In [ ]:
pokemon.tail(10)

In [ ]:
pokemon.sample(10)

A partir de esto, se puede observar que existen múltiples versiones del mismo Pokémon, como en el caso de CharizardMega Charzard X/Y y Charizard, que tienen el mismo número. Sin embargo, X e Y tienen un total más alto que el Charizard original, pero igual entre sí. Esto explica por qué, aunque hay 800 entradas, el número solo llega a 720 y el total solo alcanza 780.

Las demás columnas aparecen como se esperaba, siendo la columna numérica reflejo de valores acordes a lo que esperaríamos. La columna "Generation" representa la generación en la que se introdujo el Pokémon, y "Legendary" actúa como un valor booleano para indicar si el Pokémon es legendario o no.

Podemos realizar un análisis exploratorio de los datos antes de intentar analizar las anomalías, examinando factores como contar cuántos son legendarios, la distribución a lo largo de las generaciones y el número de tipos diferentes de Pokémon.

El primer análisis a llevar a cabo es ver si existen anomalías univariadas, es decir, si podemos identificar anomalías que ocurran en una sola variable.

Para esto, utilizando el conjunto de datos de Pokémon anterior, el enfoque aquí estará en el atributo `hp`.

Una de las primeras exploraciones a realizar consiste en trazar la distribución de la variable para determinar si la distribución está potencialmente sesgada y, como tal, si se pueden identificar anomalías de esta manera. Podemos hacer esto utilizando la biblioteca `Seaborn` de la siguiente manera:

In [ ]:
#create the displot
sns.displot(pokemon["hp"], kde = True)

#label the axis
plt.xlabel("hp", fontsize = 15)
plt.ylabel("Count", fontsize = 15)
plt.title("Pokemon HP distribution", fontsize = 15)

plt.show()

A partir de esto, podemos ver que es probable que haya algunos valores atípicos aquí, ya que la distribución tiene una asimetría positiva con algunos puntos que caen en el extremo derecho de la distribución. Podemos encontrar el valor exacto de esta asimetría y la curtosis utilizando:

In [ ]:
print(f"skewness: {pokemon['hp'].skew():.2f}")
print(f"Kurtosis: {pokemon['hp'].kurt():.2f}")

A partir de esto, podemos ver que la asimetría es mayor que $1$, lo que sugiere un alto grado de asimetría positiva y el valor de la curtosis mayor que $3$ sugiere la presencia de valores atípicos significativos en los datos.

Por lo tanto, podemos tener confianza en que existe un grado de asimetría en los datos y, por lo tanto, algunos posibles valores atípicos. Utilizando la función de distribución, podríamos sugerir que cualquier valor mayor a $150 hp$ podría considerarse fuera de la distribución existente y, por lo tanto, clasificarse como valores atípicos. Podemos extraer esta información de la siguiente manera:

In [ ]:
pokemon_above_150_HP = pokemon[pokemon["hp"] > 150]
pokemon_above_150_HP

Esto sugiere que estos 8 Pokémon podrían ser valores atípicos potenciales en términos de `hp`. Sin embargo, esto se basa solo en una inspección visual.

Un método alternativo para detectar valores atípicos en esto podría ser a través de un diagrama de `boxplot`, que muestra la media, los cuartiles inferior y superior, y se extiende a puntos que se encuentran dentro de $1.5 IQRs$ (Rango Intercuartílico) del cuartil inferior y superior. Las observaciones que caen fuera de este rango se muestran de manera independiente y se sugiere que son valores atípicos potenciales en los datos:

In [ ]:
#create the boxplot
ax = sns.boxplot(x = pokemon["hp"])

#add labels to the plot
ax.set_xlabel("HP", fontsize = 15)
ax.set_ylabel("Variable", fontsize = 15)
ax.set_title("pokemon HP boxplot", fontsize =20, pad = 20)

plt.show()

Esto sugiere de manera más clara que existen varios valores atípicos en los datos, incluyendo aquellos tanto en el extremo superior como en el inferior de la escala, mientras que al examinar la distribución original solo sugería valores atípicos en el extremo superior de la escala.

Podemos extraer estos puntos calculando el Rango Intercuartílico y luego extrayendo los puntos por encima y por debajo de este valor de la siguiente manera:

In [ ]:
#extract the upper and lower quantiles
pokemon_HP_lq = pokemon["hp"].quantile(0.25)
pokemon_HP_uq = pokemon["hp"].quantile(0.75)
#extract the inter quartile range
pokemon_HP_iqr = pokemon_HP_uq - pokemon_HP_lq

#get the upper and lower bounds
lower_bound = pokemon_HP_lq - 1.5*pokemon_HP_iqr
upper_bound = pokemon_HP_uq + 1.5*pokemon_HP_iqr

#extract values outside these bounds 
Pokemon_IQR_outliers = pokemon[(pokemon.hp <= lower_bound) | (pokemon.hp >= upper_bound)]
Pokemon_IQR_outliers

Esto sugiere, por lo tanto, significativamente más valores atípicos que el análisis anterior, ya que los umbrales aparecen alrededor de $1$ y $125$ `hp` en este caso. De esta manera, obtenemos un ($1$) Pokémon que cae por debajo del umbral inferior y $23$ que caen por encima del umbral superior.

Podríamos extender este análisis examinando valores extremos aún mayores como umbrales, bajo la suposición de que no habría tantos valores atípicos. En este caso, en lugar de multiplicar el Rango Intercuartílico por $1.5$, podemos multiplicarlo por $3$ de la siguiente manera:

In [ ]:
lower_bound_extreme = pokemon_HP_lq - 3*pokemon_HP_iqr
upper_bound_extreme = pokemon_HP_uq + 3*pokemon_HP_iqr

Pokemon_extreme_IQR_outliers = pokemon[(pokemon.hp <= lower_bound_extreme) | (pokemon.hp >= upper_bound_extreme)]
Pokemon_extreme_IQR_outliers

Esto sugiere que en lugar de tener $24$ valores atípicos, ahora solo tenemos $6$ valores atípicos y todos ellos tienen `hp` en la parte superior de la distribución de los datos.

Otro método sigue la lógica del *Z-score*. El gráfico de distribución anterior muestra una distribución aproximadamente normal, pero con una cola larga como ya se mencionó. Podemos examinar esto más de cerca en términos de la desviación estándar de los datos lejos de la media, considerando como valores extremos aquellos con un *Z-score* absoluto mayor que $3$, como sigue.

Este puntaje se calcula de la siguiente manera:

$$Z=\frac{x-\mu}{\sigma}$$

que podemos implementar como:

In [ ]:
#calculate the Z score
pokemon["HP_z"] = (pokemon["hp"] - pokemon["hp"].mean())/pokemon["hp"].std()

#show the distribution plot
sns.displot(pokemon["HP_z"],
           kde = True)

#add the labels
plt.xlabel("HP", fontsize = 15)
plt.ylabel("Count", fontsize = 15)
plt.title("Pokemon HP distribution", fontsize = 15)

plt.show()

Donde los valores atípicos pueden ser extraídos como cualquier valor que tenga un valor absoluto de *Z-score* mayor que $3$. Lo cual puede ser extraído de la siguiente manera:

In [ ]:
pokemon_HP_outliers = pokemon[abs(pokemon["HP_z"]) >= 3]
pokemon_HP_outliers

Esto extrae solo $11$ valores atípicos del conjunto de datos y la mayoría de ellos son aquellos que tienen valores de `hp` extremadamente altos en lugar de aquellos con valores extremadamente bajos.

El último método que potencialmente se puede llevar a cabo, nuevamente a través de la inspección visual, es poder agrupar los datos utilizando el algoritmo de agrupamiento *K-Means* y luego utilizar esto para identificar posibles valores atípicos en función de los valores de los grupos. Podemos hacer esto de la siguiente manera:

In [ ]:
#convert the HP values to a float to be able to use numpy 
HP_raw = pokemon["hp"].values.astype("float")
#use the kmeans function from scipy
centroids, avg_distance = kmeans(HP_raw, 4)
#extract the groups from the data
groups, cdit = vq(HP_raw, centroids)

#plot the results
#assign groups back to the dataframe
pokemon["HP_groups"] = groups

#plot the scatter plot
fig = px.scatter(pokemon, x = "hp", y = pokemon.index,
                color = "HP_groups",
                hover_name = "name")
#add a title
fig.update_layout(title = "K-Means outlier detection",
                 title_x = 0.5)
#show the plot
fig.show()

Podemos observar que con cuatro grupos, posibles valores atípicos podrían ser detectados en el grupo $0$ o en el grupo $3$ en términos de valores atípicos potenciales que ocurren en la parte superior y en la parte inferior de la distribución.

El hecho de que estos métodos sugieran diferentes valores atípicos potenciales demuestra que la detección de valores atípicos no es una ciencia exacta y se deben tomar decisiones de juicio en términos de si estos son valores atípicos o no, dependiendo del propósito del análisis.

## Análisis Multivariado

El análisis multivariable intenta identificar anomalías basadas en más de una variable. Por ejemplo, podemos examinar si existen anomalías en las categorías de ataque y defensa de los Pokémon. Como antes, podemos examinar esto visualmente, lo cual con dos variables es relativamente fácil en un gráfico de dispersión, pero se vuelve más complicado a medida que hay más variables debido a las mayores dimensiones:

In [ ]:
#create the base axis
fig, ax = plt.subplots(1,1, figsize = (8,10))

#plot teh scatter plot
ax.scatter(pokemon["attack"], pokemon["defense"])

#add labels
ax.set_xlabel("Attack", fontsize = 20, labelpad = 20)
ax.set_ylabel("Defense", fontsize = 20, labelpad = 20)
ax.set_title("Pokemon Attack and Defense scatter plot", fontsize = 20, pad = 20)
#alter the tick parametes
ax.tick_params(axis = "both", labelsize = 20)

Como podemos ver en este gráfico, generalmente existe una relación lineal positiva entre la defensa y el ataque de los Pokémon, pero pareciera haber algunos valores atípicos potenciales con alta defensa y bajo ataque, o viceversa, que se desvían de la mayoría de los puntos. La clave es poder identificar estos valores atípicos potenciales en el conjunto de datos, para lo cual existen varios algoritmos diferentes que se pueden utilizar.

Una diferencia clave entre esto y el análisis univariado es que ahora tenemos múltiples variables para analizar, por lo que donde anteriormente podría haber habido un valor atípico en una sola dimensión, esto no significa necesariamente que también sea un valor atípico en ambas dimensiones.

Podemos examinar esto nuevamente utilizando diagramas de caja al igual que para la variable única para ver si existen diferencias en las distribuciones:

In [ ]:
ax = sns.boxplot(data = pokemon[["attack", "defense"]], orient = "h", palette = "Set2")

ax.set_xlabel("Value", fontsize = 20, labelpad = 20)
ax.set_ylabel("Attributes", fontsize = 20, labelpad = 20)
ax.set_title("Boxplot of pokemon Attack \nand Defense attributes", fontsize = 20,
            pad = 20)
ax.tick_params(which = "both", labelsize = 15)

Para cada categoría, esto sugiere varios valores atípicos en el conjunto de datos, los cuales podemos examinar utilizando el rango del *IQR* (Rango Intercuartílico) como lo hicimos en el análisis univariado nuevamente, y extraer esto del conjunto de datos original:

In [ ]:
#create a function to calculate IQR bounds
def IQR_bounds(dataframe, column_name, multiple):
    """Extract the upper and lower bound for outlier detection using IQR
    
    Input:
        dataframe: Dataframe you want to extract the upper and lower bound from
        column_name: column name you want to extract upper and lower bound for
        multiple: The multiple to use to extract this
        
    Output:
        lower_bound = lower bound for column
        upper_bound = upper bound for column"""
    
    #extract the quantiles for the column
    lower_quantile = dataframe[column_name].quantile(0.25)
    upper_quantile = dataframe[column_name].quantile(0.75)
    #cauclat IQR
    IQR = upper_quantile - lower_quantile
    
    #extract lower and upper bound
    lower_bound = lower_quantile - multiple * IQR
    upper_bound = upper_quantile + multiple * IQR
    
    #retrun these values
    return lower_bound, upper_bound

In [ ]:
#set the columns we want
columns = ["attack", "defense"]
#create a dictionary to store the bounds
column_bounds = {}

#iteratre over each column to extract bounds
for column in columns:
    #extract normal and extreme bounds
    lower_bound, upper_bound =  IQR_bounds(pokemon, column, 1.5)
    lower_bound_extreme, upper_bound_extreme = IQR_bounds(pokemon, column, 3)
    #send them to the dictionary
    column_bounds[column] = [lower_bound, upper_bound,
                            lower_bound_extreme, upper_bound_extreme]

#create the normal dataframe
pokemon_IQR_AD = pokemon[(pokemon["attack"] < column_bounds["attack"][0]) | 
                         (pokemon["attack"] > column_bounds["attack"][1]) |
                         (pokemon["defense"] < column_bounds["defense"][0]) | 
                         (pokemon["defense"] > column_bounds["defense"][1])
                        ]
#create the extreme dataframe
pokemon_IQR_AD_extreme = pokemon[(pokemon["attack"] < column_bounds["attack"][2]) |
                         (pokemon["attack"] > column_bounds["attack"][3]) |
                         (pokemon["defense"] < column_bounds["defense"][2]) | 
                         (pokemon["defense"] > column_bounds["defense"][3])
                         ]

In [ ]:
#print the normal dataframe
pokemon_IQR_AD 

In [ ]:
#print the extreme dataset
pokemon_IQR_AD_extreme

Podemos ver esto en términos del gráfico original al reintegrar esta información en el marco de datos original:

In [ ]:
#create the normal dataframe
pokemon["IQR_AD"] = ((pokemon["attack"] < column_bounds["attack"][0]) | 
                         (pokemon["attack"] > column_bounds["attack"][1]) |
                         (pokemon["defense"] < column_bounds["defense"][0]) | 
                         (pokemon["defense"] > column_bounds["defense"][1])
                    )
#create the extreme dataframe
pokemon["IQR_AD_extreme"] = ((pokemon["attack"] < column_bounds["attack"][2]) |
                         (pokemon["attack"] > column_bounds["attack"][3]) |
                         (pokemon["defense"] < column_bounds["defense"][2]) | 
                         (pokemon["defense"] > column_bounds["defense"][3])
                            )

In [ ]:
pokemon["IQR_AD"] = pokemon["IQR_AD"].apply(lambda x: str(1) if x == False else str(-1))
pokemon["IQR_AD_extreme"] = pokemon["IQR_AD_extreme"].apply(lambda x: str(1) if x == False else str(-1))

In [ ]:
#plot teh scatter plot
fig = px.scatter(pokemon, x = "attack", y = "defense",
          color = "IQR_AD", 
          hover_name = "name")
fig.update_layout(title = "IQR outlier detection",
                 title_x = 0.5)
fig.show()

In [ ]:
#plot teh scatter plot
fig = px.scatter(pokemon, x = "attack", y = "defense",
          color = "IQR_AD_extreme", 
          hover_name = "name")
fig.update_layout(title = "IQR extreme outlier detection",
                 title_x = 0.5)
fig.show()

De lo cual podemos observar que este método ha encontrado valores atípicos en función de cada variable individual en lugar de necesariamente la combinación entre ellas. Por lo tanto, podemos mejorar este método utilizando diferentes algoritmos para examinar cualquier valor atípico en los datos.

## Algoritmos

### Isolation Forest

*Isolation Forest* es un buen punto de partida para la detección de anomalías, especialmente en conjuntos de datos de dimensiones más altas.

Es un método de conjunto de árboles que proviene del grupo de algoritmos en conjunto de la biblioteca `sklearn`. Está construido sobre la base de árboles de decisión, al igual que [`random forest`](https://en.wikipedia.org/wiki/Random_forest), donde los árboles se dividen primero seleccionando aleatoriamente una característica y luego seleccionando un valor de división aleatorio entre el valor máximo y mínimo de la característica seleccionada.

En principio, las anomalías son menos frecuentes que las observaciones regulares y son diferentes en términos de sus valores. Por esta razón, al utilizar particiones aleatorias, deberían ser identificadas como más cercanas a la raíz del árbol, con menos divisiones necesarias, que los datos no anómalos. Por lo tanto, a diferencia de otros algoritmos en los que se centra en los datos normales y luego se identifican las anomalías, aquí se enfoca en identificar inicialmente las anomalías y luego los datos normales en función de esto.

Sin embargo, para esto, primero debemos especificar el parámetro de contaminación, que se utiliza para indicar cuántos de los datos se esperan que sean anomalías. En nuestro caso, podemos establecerlo en $0.02$ para indicar que creemos que el $2\%$ de los datos pueden ser anómalos, lo que significa que deberíamos obtener $16$ puntos de datos anómalos a partir de este método:

In [ ]:
from sklearn.ensemble import IsolationForest

#create the method instance
isf = IsolationForest(n_estimators = 100, random_state = 42, contamination = 0.02)
#use fit_predict on the data as we are using all the data
preds = isf.fit_predict(pokemon[["attack", "defense"]])
#extract outliers from the data
pokemon["iso_forest_outliers"] = preds
pokemon["iso_forest_outliers"] = pokemon["iso_forest_outliers"].astype(str)
#extract the scores from the data in terms of strength of outlier
pokemon["iso_forest_scores"] = isf.decision_function(pokemon[["attack", "defense"]])

#print how many outliers the data suggests
print(pokemon["iso_forest_outliers"].value_counts())

A partir de esto, se puede observar que tenemos $16$ valores atípicos como se esperaba. ¿Cómo se ve esto en términos del gráfico subyacente?

In [ ]:
#this plot will be repeated so it is better to create a function
def scatter_plot(dataframe, x, y, color, title, hover_name):
    """Create a plotly express scatter plot with x and y values with a colour
    
    Input:
        dataframe: Dataframe containing columns for x, y, colour and hover_name data
        x: The column to go on the x axis
        y: Column name to go on the y axis
        color: Column name to specify colour
        title: Title for plot
        hover_name: column name for hover
        
    Returns:
        Scatter plot figure
    """
    #create the base scatter plot
    fig = px.scatter(dataframe, x = x, y=y,
                    color = color,
                     hover_name = hover_name)
    #set the layout conditions
    fig.update_layout(title = title,
                     title_x = 0.5)
    #show the figure
    fig.show()

In [ ]:
#create scatter plot
scatter_plot(pokemon, "attack", "defense", "iso_forest_outliers",
             "Isolation Forest Outlier Detection",
            "name")

Podemos ver que esto produce valores atípicos similares a los del análisis del *IQR* presentado anteriormente, pero que los límites de decisión no son tan lineales como lo eran para el *IQR*, y aunque los puntos habían sido considerados como valores atípicos antes, no necesariamente lo son en este conjunto de datos.

Podemos observar cómo se ha tomado esta decisión trazando las puntuaciones (scores) del conjunto de datos. Cuanto más bajo sea el puntaje, es más probable que se considere un valor atípico, mientras que cuanto más alto sea el puntaje, menos probable será:

In [ ]:
#create the same plot focusing on the scores from the dataset
scatter_plot(pokemon, "attack", "defense", "iso_forest_scores",
             "Isolation Forest Outlier Detection Scores",
            "name")

A partir de esto, podemos ver que los puntos centrales en el conjunto de datos son menos propensos a ser considerados valores atípicos, mientras que aquellos en el borde son más propensos a ser vistos como valores atípicos, como se refleja en el gráfico de valores atípicos mencionado anteriormente.

Podemos observar la distribución de estos puntajes de la siguiente manera:

In [ ]:
#create the distribution plot
sns.displot(pokemon["iso_forest_scores"],color='red',label='if',
           kde = True);

#set the title
plt.title('Distribution of Isolation Forest Scores', fontsize = 15, loc='center')
plt.xlabel("Isolation Forest Scores", fontsize = 15)
plt.ylabel("Count", fontsize = 15)

#show the result
plt.show()

Los beneficios de este método son que es efectivo cuando no se pueden asumir las distribuciones de valores, tiene pocos parámetros y la implementación en `Scikit-Learn` es fácil de usar y entender.

Sin embargo, visualizar los resultados y/o el mecanismo subyacente puede ser complicado y con conjuntos de datos grandes, el entrenamiento puede ser largo y computacionalmente costoso.

### Local outlier Factor (LoF)

Otro algoritmo que se puede utilizar para identificar posibles anomalías en el conjunto de datos es el algoritmo [Local Outlier Factor](https://en.wikipedia.org/wiki/Local_outlier_factor) (LOF), que también forma parte de `SciKit Learn`.

El *Local Outlier Factor (Factor Local de Anomalía, LOF)* es un cálculo que examina los vecinos de un punto para encontrar su densidad y luego compara esto con la densidad de los vecinos. Si la densidad de un punto es mucho menor que la de sus vecinos, se sugiere que es un valor atípico. Esto significa que es un método local, ya que el puntaje depende de cuán aislado está el punto en comparación con sus propios vecinos.

El punto clave para esto es el número de vecinos que se especifica para comparar y la métrica que se utiliza para calcular la densidad. Para esto, el número predeterminado de vecinos es $20$ (aunque si la proporción de valores atípicos es mayor al $10\%$, $n$ debería ser mayor) y la métrica predeterminada es la [distancia Minkowski](https://en.wikipedia.org/wiki/Minkowski_distance) (que generaliza tanto la distancia euclidiana como la [distancia de Manhattan](https://en.wikipedia.org/wiki/Taxicab_geometry)).

El beneficio de este algoritmo es que puede tener en cuenta tanto las propiedades locales como globales del conjunto de datos, ya que se enfoca en cuán aislada está la muestra con respecto al vecindario circundante.

Por lo tanto, podemos aplicarlo de la siguiente manera:

In [ ]:
#import the algorithm
from sklearn.neighbors import LocalOutlierFactor

#initialise the algorithm
lof = LocalOutlierFactor(n_neighbors = 20)
#fit it to the training data, since we don't use it for novelty than this is fine
y_pred = lof.fit_predict(pokemon[["attack", "defense"]])

#extract the predictions as strings
pokemon["lof_outliers"] = y_pred.astype(str)
#print the number of outliers relative to non-outliers
print(pokemon["lof_outliers"].value_counts())
#extract the outlier scores
pokemon["lof_scores"] = lof.negative_outlier_factor_

Usar el valor predeterminado de `n_neighbors = 20` nos da $39$ valores atípicos potenciales, que se pueden comparar con el algoritmo anterior al examinar dónde se detectan estos valores atípicos:

In [ ]:
scatter_plot(pokemon, "attack", "defense", "lof_outliers",
             "Local Outlier Factor Outlier Detection",
            "name")

In [ ]:
scatter_plot(pokemon, "attack", "defense", "lof_scores",
             "Local Outlier Factor scores",
            "name")

Podemos ver en estos gráficos que, al igual que con el algoritmo anterior, los valores atípicos se detectan en el borde de la masa principal. Esto se debe a la énfasis en la densidad, ya que solo hay una masa principal de puntos en este conjunto de datos, y es probable que los puntos que se encuentren fuera de esta región se identifiquen como valores atípicos. Esto significa que se detectan muchos más valores atípicos utilizando este algoritmo que el anterior.

Nuevamente, también podemos observar la distribución de los puntajes en este gráfico:

In [ ]:
#create the distribution plot
sns.displot(pokemon["lof_scores"],color='red',label='if',
           kde = True);

#set the title
plt.title("Distribution of Local Outlier Factor Scores", fontsize = 15, loc='center', pad = 20)
plt.xlabel("Local Outlier Factor", fontsize = 15, labelpad = 20)
plt.ylabel("Count", fontsize = 15, labelpad = 20)

#show the result
plt.show()

### DBSCAN

*DBScan* es un algoritmo de agrupamiento comúnmente utilizado que también puede detectar valores atípicos en un conjunto de datos. Funciona de manera similar al Factor Local de Anomalía en el sentido de que examina los vecinos de un punto, pero se comporta ligeramente diferente. Esto se debe a que selecciona aleatoriamente un punto que aún no esté asignado a un clúster ni designado como valor atípico, y determina si es un punto central viendo si hay al menos un número mínimo dado de muestras dentro de una distancia dada. Si es así, entonces se designa como un punto central junto con todos los puntos a los que se puede llegar directamente desde ese punto. Esto se repite hasta que se identifica el borde del clúster, donde no hay más puntos dentro de la distancia epsilon del clúster.

Si un punto no cae dentro de ninguno de los clústeres potenciales, entonces se considera un valor atípico en base a que no encaja en la densidad o clúster de puntos existente.

Por lo tanto, esto se puede implementar de la siguiente manera:

In [ ]:
#import the algorithm
from sklearn.cluster import DBSCAN

#initiate the algorithm
#set the distance to 20, and min_samples as 5
outlier_detection = DBSCAN(eps = 20, metric = "euclidean", min_samples = 10, n_jobs = -1)
#fit_predict the algorithm to the existing data
clusters = outlier_detection.fit_predict(pokemon[["attack", "defense"]])

#extract the labels from the algorithm
pokemon["dbscan_outliers"] = clusters
#label all others as inliers 
pokemon["dbscan_outliers"] = pokemon["dbscan_outliers"].apply(lambda x: str(1) if x>-1 else str(-1))
#print the vaue counts
print(pokemon["dbscan_outliers"].value_counts())

Los resultados pueden ser representados gráficamente de la siguiente manera:

In [ ]:
scatter_plot(pokemon, "attack", "defense", "dbscan_outliers",
             "DBScan Outlier Detection",
            "name")

La diferencia clave aquí es que solo aquellos en los bordes superiores de ambos atributos fueron seleccionados como valores atípicos por este algoritmo. Esto se debe a que se han extendido más allá del alcance de los puntos centrales, mientras que aquellos en los bordes inferiores de la defensa y también de ambos estarían justo dentro del alcance mínimo de los puntos centrales del clúster.

El beneficio de este método es que se puede utilizar cuando no se puede asumir la distribución de valores en el espacio de características, ha sido implementado en `SciKit Learn` y es intuitivo de entender.

Los problemas de este método son que seleccionar los parámetros óptimos puede ser difícil, es no supervisado y puede tener dificultades en conjuntos de datos de alta dimensión debido al uso de la distancia para clasificar diferentes puntos, lo cual se vuelve menos claro en un espacio de alta dimensión, y tiene dificultades para ser utilizado en capacidad predictiva.

### One Class SVM

Se puede ajustar un clasificador de una sola clase en un conjunto de datos de entrenamiento que solo tiene ejemplos de la clase normal, pero también se puede utilizar para todos los datos, como en este caso. Una vez preparado el modelo, se puede usar para clasificar nuevos ejemplos como normales o valores atípicos esperados. La diferencia principal con una máquina de soporte vectorial estándar es que se ajusta de manera no supervisada y no proporciona la capacidad normal de ajuste de hiperparámetros como una *SVM* normal. En su lugar, proporciona un parámetro `nu` que controla la sensibilidad de los vectores de soporte y debe ajustarse para aproximar la proporción de valores atípicos en los datos.

In [ ]:
#import the required library
from sklearn import svm

#initiate the model
svm_model = svm.OneClassSVM(nu = 0.2, kernel = "rbf", gamma = "auto")
#apply the model to the data
outliers = svm_model.fit_predict(pokemon[["attack", "defense"]])

#extract the labels
pokemon["ocsvm_outliers"] = outliers
#change the labels
pokemon["ocsvm_outliers"] = pokemon["ocsvm_outliers"].apply(lambda x: str(-1) if x == -1 else str(1))
#extract the score
pokemon["ocsvm_scores"] = svm_model.score_samples(pokemon[["attack", "defense"]])
#print the value counts for inlier and outliers
print(pokemon["ocsvm_outliers"].value_counts())

Utilizando el parámetro `nu`, el mejor ajuste para esto resulta en un total de $279$ valores atípicos en el conjunto de datos. Podemos visualizar esto como antes:

In [ ]:
scatter_plot(pokemon, "attack", "defense", "ocsvm_outliers",
             "One Class SVM Outlier Detection",
            "name")

De lo cual podemos ver que el modelo ha tenido un desempeño bastante pobre, con una mezcla tanto de valores normales como de valores atípicos en la distribución central de los datos. Esto sugiere, por lo tanto, un ajuste deficiente del modelo, ya que va en contra de lo que supondríamos que son valores atípicos en estos datos.

### Elliptic envelope

El [Elliptic envelope](https://en.wikipedia.org/wiki/Elliptical_distribution) ("Envolvente elíptico") es otro algoritmo de detección de anomalías, pero uno que asume una distribución gaussiana como parte de los datos. Esto funciona creando un área elíptica imaginaria alrededor de un conjunto de datos dado, donde los valores que caen dentro de esas elipses se consideran datos normales y cualquier cosa que caiga fuera de eso se asume que son valores atípicos.

La implementación de este modelo, al igual que el algoritmo *Isolation Forest*, requiere que el parámetro de contaminación se establezca previamente para sugerir cuántos valores atípicos esperar para el modelo. Como antes, podemos establecer esto en $0.02$.

Este es otro método que está implementado en `sklearn` y, por lo tanto, se puede implementar fácilmente de manera similar a como se hizo antes:

In [ ]:
#import the necessary library and functionality
from sklearn.covariance import EllipticEnvelope

#create the model, set the contamination as 0.02
EE_model = EllipticEnvelope(contamination = 0.02)
#implement the model on the data
outliers = EE_model.fit_predict(pokemon[["attack", "defense"]])

#extract the labels
pokemon["EE_outliers"] = outliers
#change the labels
pokemon["EE_outliers"] = pokemon["EE_outliers"].apply(lambda x: str(-1) if x == -1 else str(1))
#extract the score
pokemon["EE_scores"] = EE_model.score_samples(pokemon[["attack", "defense"]])
#print the value counts for inlier and outliers
print(pokemon["EE_outliers"].value_counts())

In [ ]:
#plot the results
scatter_plot(pokemon, "attack", "defense", "EE_outliers",
             "Elliptic Envelope Outlier Detection",
            "name")

In [ ]:
#plot the scores
scatter_plot(pokemon, "attack", "defense", "EE_scores",
             "Elliptic Envelope scores",
            "name")

Podemos ver cómo se desempeña en el conjunto de datos por el hecho de que las puntuaciones son mayores cerca del centro de la distribución, con las elipses esencialmente dibujadas alrededor de esos puntos. La puntuación en sí es la *distancia de Mahalanobis negativa*, que es la inversa de una medida de distancia entre el punto y una distribución (en esencia, las elipses). Por lo tanto, los puntos que están más alejados de las elipses en este gráfico obtienen puntuaciones más bajas.

Una diferencia principal en los valores atípicos detectados por este algoritmo en comparación con antes es que los puntos hacia el centro izquierdo no se identifican como valores atípicos aquí, lo que sugiere que aquí se han contado dentro de las elipses y, por lo tanto, es diferente principalmente debido a las suposiciones del modelo.

Las dificultades con este método incluyen el requisito de normalidad en la distribución de las variables y que no conocemos el valor exacto del parámetro de contaminación. Sin embargo, para ayudar con esto, cuando el modelo se implementa en los datos, asumiendo que se cumple la suposición de normalidad, se puede realizar un análisis univariado en las puntuaciones del modelo para identificar en qué medida se pueden identificar anomalías.

### Ensemble

Una forma de aumentar la confianza en estos métodos es no solo utilizar un solo método, sino usar una combinación de métodos para poder determinar valores atípicos. Si bien el método *One Class SVM* tuvo un rendimiento bastante pobre en el conjunto de datos, en su lugar podemos utilizar los resultados de los otros algoritmos para combinarlos y crear un rango de confianza en la capacidad de identificar cualquier valor atípico.

Podemos hacer esto de la siguiente manera:

In [ ]:
#extract the sum of the outlier count
pokemon['outliers_sum'] = (pokemon['iso_forest_outliers'].astype(int)+
                           pokemon['lof_outliers'].astype(int)+
                           pokemon['dbscan_outliers'].astype(int)+
                          pokemon['EE_outliers'].astype(int))
#print the value counts for each scale
print(pokemon["outliers_sum"].value_counts())

In [ ]:
scatter_plot(pokemon, "attack", "defense", "outliers_sum",
             "Ensemble outlier detection",
            "name")

A partir de esto, podemos delinear claramente cualquier valor atípico del conjunto de datos, con aquellos que son identificados como valores atípicos en los tres algoritmos teniendo una suma total de $-3$. A partir de esto, podemos estar seguros de que se han detectado valores atípicos en el conjunto de datos.

Podemos examinar qué Pokémon están asociados con estos valores atípicos extrayendo los Pokémon en los que estamos interesados:

In [ ]:
pokemon.loc[pokemon['outliers_sum']==-4]['name']